In [207]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import random
import os

In [208]:
pd.__version__

'1.4.4'

Se a gente lê o arquivo de 3GB inteiro, vai dar problema. Aqui tem uma receita para ler somente x%: https://www.kaggle.com/questions-and-answers/53925 

## Selecionar apenas uma amostra aleatória de 1% dos alunos

In [209]:
def skip(i,fraction=0.01):
    if i == 0:
        return False  # precisamos manter o row=0 para ter o header
    else:
        return random.random() > fraction

[skip(i) for i in range(10)]

[False, True, True, True, True, True, True, True, True, True]

In [210]:
%time df = pd.read_csv('../inep/enem/2010/DADOS/MICRODADOS_ENEM_2010.csv',skiprows = skip, encoding='latin1',sep=";")

CPU times: user 21.8 s, sys: 48 ms, total: 21.9 s
Wall time: 39.4 s


In [211]:
for col in df.columns:
    print(col)

NU_INSCRICAO
NU_ANO
TP_FAIXA_ETARIA
TP_SEXO
TP_ESTADO_CIVIL
TP_COR_RACA
TP_ST_CONCLUSAO
TP_ENSINO
CO_MUNICIPIO_ESC
NO_MUNICIPIO_ESC
CO_UF_ESC
SG_UF_ESC
TP_DEPENDENCIA_ADM_ESC
TP_LOCALIZACAO_ESC
TP_SIT_FUNC_ESC
IN_CERTIFICADO
CO_MUNICIPIO_PROVA
NO_MUNICIPIO_PROVA
CO_UF_PROVA
SG_UF_PROVA
TP_PRESENCA_CN
TP_PRESENCA_CH
TP_PRESENCA_LC
TP_PRESENCA_MT
CO_PROVA_CN
CO_PROVA_CH
CO_PROVA_LC
CO_PROVA_MT
NU_NOTA_CN
NU_NOTA_CH
NU_NOTA_LC
NU_NOTA_MT
TX_RESPOSTAS_CN
TX_RESPOSTAS_CH
TX_RESPOSTAS_LC
TX_RESPOSTAS_MT
TP_LINGUA
TX_GABARITO_CN
TX_GABARITO_CH
TX_GABARITO_LC
TX_GABARITO_MT
TP_STATUS_REDACAO
NU_NOTA_COMP1
NU_NOTA_COMP2
NU_NOTA_COMP3
NU_NOTA_COMP4
NU_NOTA_COMP5
NU_NOTA_REDACAO
Q01
Q02
Q03
Q04
Q05
Q06
Q07
Q08
Q09
Q10
Q11
Q12
Q13
Q14
Q15
Q16
Q17
Q18
Q19
Q20
Q21
Q22
Q23
Q24
Q25
Q26
Q27
Q28
Q29
Q30
Q31
Q32
Q33
Q34
Q35
Q36
Q37
Q38
Q39
Q40
Q41
Q42
Q43
Q44
Q45
Q46
Q47
Q48
Q49
Q50
Q51
Q52
Q53
Q54
Q55
Q56
Q57


In [212]:
df

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,...,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Q55,Q56,Q57
0,200000000006,2010,2,F,0,1,2,1.0,3304557.0,RIO DE JANEIRO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200000000050,2010,4,M,0,3,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200000000086,2010,10,F,0,3,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200000000167,2010,3,F,0,1,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200000000181,2010,4,F,0,1,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46005,200004611098,2010,13,F,0,1,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46006,200004611102,2010,5,F,0,1,3,NaN,NaN,NaN,...,4.0,0.0,0.0,0.0,0.0,D,5.0,5.0,5.0,5.0
46007,200004611239,2010,12,F,1,2,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46008,200004611554,2010,16,M,1,2,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [213]:
#df.describe()

In [214]:
#df.sample(5)

In [215]:
df["TP_PRESENCA_CN"].value_counts()

1    33844
0    12108
2       58
Name: TP_PRESENCA_CN, dtype: int64

## Limpeza dos dados, selecionando apenas alunos que não faltaram na prova

In [216]:

def filterdf(df):
    if "IN_TREINEIRO" in df.columns:
        df = df[df["TP_PRESENCA_CN"] == 1]  # presente nas 4 provas
        df = df[df["TP_PRESENCA_CH"] == 1]
        df = df[df["TP_PRESENCA_LC"] == 1]
        df = df[df["TP_PRESENCA_MT"] == 1]
        df = df[(df["IN_TREINEIRO"] == 0) | (df["IN_TREINEIRO"].isna())]  # não é treineiro ou não existe 
        df = df[df["TP_ST_CONCLUSAO"].isin([1, 2])]  # afirma que concluiu ou vai concluir EM em 2019
        # não queremos as notas == 0
        df = df.query("NU_NOTA_CH != 0 and NU_NOTA_CN != 0 and NU_NOTA_LC != 0 and NU_NOTA_MT != 0") 
    return df

dff = filterdf(df)
dff

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,...,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Q55,Q56,Q57
0,200000000006,2010,2,F,0,1,2,1.0,3304557.0,RIO DE JANEIRO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200000000050,2010,4,M,0,3,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200000000086,2010,10,F,0,3,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200000000167,2010,3,F,0,1,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200000000181,2010,4,F,0,1,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46005,200004611098,2010,13,F,0,1,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46006,200004611102,2010,5,F,0,1,3,NaN,NaN,NaN,...,4.0,0.0,0.0,0.0,0.0,D,5.0,5.0,5.0,5.0
46007,200004611239,2010,12,F,1,2,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46008,200004611554,2010,16,M,1,2,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [217]:
import inspect
filter_conditions = inspect.getsource(filterdf)
print(filter_conditions)

def filterdf(df):
    if "IN_TREINEIRO" in df.columns:
        df = df[df["TP_PRESENCA_CN"] == 1]  # presente nas 4 provas
        df = df[df["TP_PRESENCA_CH"] == 1]
        df = df[df["TP_PRESENCA_LC"] == 1]
        df = df[df["TP_PRESENCA_MT"] == 1]
        df = df[(df["IN_TREINEIRO"] == 0) | (df["IN_TREINEIRO"].isna())]  # não é treineiro ou não existe 
        df = df[df["TP_ST_CONCLUSAO"].isin([1, 2])]  # afirma que concluiu ou vai concluir EM em 2019
        # não queremos as notas == 0
        df = df.query("NU_NOTA_CH != 0 and NU_NOTA_CN != 0 and NU_NOTA_LC != 0 and NU_NOTA_MT != 0") 
    return df



## Salvar a amostra filtrada em disco

In [218]:

fn = "data/enem19-1perc"
dff.to_csv(f'{fn}.csv',index=False)
with open(fn+'-filter_conditions.txt','w') as f:
    f.write(filter_conditions)

## Selecionar apenas as colunas que desejamos trabalhar

In [219]:

def selecao(dff):
    df_selecao = dff[["NU_INSCRICAO","NU_ANO","CO_MUNICIPIO_RESIDENCIA","NO_MUNICIPIO_RESIDENCIA","CO_UF_RESIDENCIA","SG_UF_RESIDENCIA","TP_COR_RACA","TP_SEXO","NU_IDADE","TP_ENSINO","TP_ESCOLA","NU_NOTA_CN","NU_NOTA_CH","NU_NOTA_MT","NU_NOTA_LC","NU_NOTA_REDACAO"]]
    return df_selecao

    

 Tentativa da função co loc
def selecao(dff):
    df_selecao = dff.loc[:,["NU_INSCRICAO","NU_ANO","CO_MUNICIPIO_RESIDENCIA","NO_MUNICIPIO_RESIDENCIA","CO_UF_RESIDENCIA","SG_UF_RESIDENCIA","TP_COR_RACA","TP_SEXO","NU_IDADE","TP_ENSINO","TP_ESCOLA","NU_NOTA_CN","NU_NOTA_CH","NU_NOTA_MT","NU_NOTA_LC","NU_NOTA_REDACAO"]]

In [220]:
df_selecionado = selecao(df)
df_selecionado

KeyError: "['CO_MUNICIPIO_RESIDENCIA', 'NO_MUNICIPIO_RESIDENCIA', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA', 'NU_IDADE', 'TP_ESCOLA'] not in index"

## Salvar colunas selecionadas

In [ ]:

df_selecao.to_csv('data/selecao.csv',index=False)
df_selecao

## Tirar alunos com nota 0 da selecao

In [ ]:

notas_selecao = df_selecao.query("NU_NOTA_CH != 0 and NU_NOTA_CN != 0 and NU_NOTA_LC != 0 and NU_NOTA_MT != 0")
notas_selecao

In [ ]:
# salvar a amostra filtrada em disco
notas_selecao.to_csv('data/enem19-sem_Zzero.csv',index=False)

In [ ]:
#df_selecao = df_selecao[df_selecao["NU_NOTA_MT"] == 0]
#df_selecao

In [ ]:
#df_selecao = df_selecao.drop(notas_zero.index)
#df_selecao

In [ ]:
#Numero pessoas que sairam da analise
len(df_selecao)-len(notas_selecao)

## Salvar dataFrame sem notas 0

In [224]:

notas_sem_0 = "data/notas_sem_0"
notas_selecao.to_csv(f'{notas_sem_0}.csv',index=False)


In [230]:
nome_completo = os.path.basename('../inep/enem/2010/DADOS/MICRODADOS_ENEM_2010.csv') #le o nome do arquivo que estamos usando
apenas_ano = nome_completo[-8:-4] #separa o ano dos microdados
apenas_ano

'2010'

In [232]:
nome_padronizado = "dados_limpos_enem_" + apenas_ano
nome_padronizado

'dados_limpos_enem_2010'

Não consigo ler o arquivo
teste = pd.read_csv("Enem/data/notas_sem_0.csv")
teste


In [ ]:
def limpo(df_selecao):
    df_selecao.to_csv('data/selecao.csv',index=False)
    print(df_selecao)
    

In [221]:
teste = os.path.abspath(df_selecao)
print(f"teste")

TypeError: expected str, bytes or os.PathLike object, not DataFrame

In [223]:
df_selecao.to_csv("df_selecao.csv", sep =";")


,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,TP_COR_RACA,TP_SEXO,NU_IDADE,TP_ENSINO,TP_ESCOLA,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_MT,NU_NOTA_LC,NU_NOTA_REDACAO
4,190001004967,2019,1506807,Santarém,15,PA,3,F,20,1.0,1,537.9,582.2,417.7,585.7,780.0
5,190001005059,2019,1506807,Santarém,15,PA,4,F,17,1.0,2,416.2,569.5,459.9,539.6,660.0
6,190001005138,2019,1508100,Tucuruí,15,PA,3,M,19,NaN,1,526.9,589.7,708.5,588.7,860.0
7,190001005147,2019,1505205,Oeiras do Pará,15,PA,3,F,37,1.0,1,376.8,502.1,392.1,546.5,620.0
8,190001005219,2019,1502954,Eldorado do Carajás,15,PA,3,F,18,1.0,1,377.6,443.8,425.0,433.5,340.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50671,190006119878,2019,2610608,Paudalho,26,PE,0,F,19,1.0,1,419.4,354.0,587.7,461.2,520.0
50672,190006119928,2019,2700300,Arapiraca,27,AL,3,F,26,1.0,1,567.1,513.7,551.0,559.2,460.0
50674,190006120038,2019,2912103,Ibicaraí,29,BA,4,F,17,NaN,2,340.9,497.7,372.7,386.8,260.0
50675,190006120062,2019,2927408,Salvador,29,BA,3,F,18,1.0,2,397.6,438.5,450.2,476.9,300.0
